# Imports

In [ ]:
import os
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import scipy as sp
import scipy.signal
import scipy.stats
from scipy.stats.mstats import gmean

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, LeaveOneOut, StratifiedKFold
from sklearn.svm import SVC

from utils import Dataset, variance_thresholding, standardize, mcc, calculate_metrics, calculate_metrics_statistics

In [ ]:
# parameters for Welch's method for estimating power spectrum

NPERSEG = 60                    # length of segment
NOVERLAP = int(0.75 * NPERSEG)  # overlap of segments
NFFT = NPERSEG                  # length of FFT
WINDOW = "hann"                 # window function type

# parameters for saving data
PROCESSED_DATA_DIR = "processed_data"
DEPRESJON_PREFIX = "manual_depresjon"
PSYKOSE_PREFIX = "manual_psykose"

# Manual feature extraction

## Helper functions

In [ ]:
def basic_data_cleaning(data: List[pd.DataFrame]) -> List[pd.DataFrame]:
    """
    Assumes DataFrames with "timestamp", "date" and "activity" columns.
    
    Performs cleaning operations:
    - assure format YYYY-MM-DD HH:MM:SS for "timestamp"
    - drop redundant "date" column
    - assure float32 format for "activity"
    
    :param data: list of DataFrames
    :returns: list of cleaned DataFrames
    """
    data = [df.copy() for df in data]  # create copy to avoid side effects
    
    for df in data:
        df["timestamp"] = pd.to_datetime(df["timestamp"],
                                         format="%Y-%m-%d %H:%M:%S")
        df.drop("date", axis=1, inplace=True)
        df["activity"] = df["activity"].astype(np.float32)
    
    return data


def get_day_part(df: pd.DataFrame, part: str) -> pd.DataFrame:
    """
    For given DataFrame with "timestamp" column returns only those rows that
    correspond to the chosen part of day.
    
    Parts are "day" and "night", defined as:
    - "day": [8:00, 21:00)
    - "night": [21:00, 8:00)
    
    :param df: DataFrame to select rows from
    :param part: part of day, either "day" or "night"
    :returns: DataFrame, subset of rows of df
    """
    if part == "day":
        df = df.loc[(df["timestamp"].dt.hour >= 8) &
                    (df["timestamp"].dt.hour < 21)]
    elif part == "night":
        df = df.loc[(df["timestamp"].dt.hour >= 21) |
                    (df["timestamp"].dt.hour < 8)]
    else:
        raise ValueError(f'Part should be "day" or "night", got "{part}"')
        
    return df


def fill_missing_activity(df: pd.DataFrame) -> pd.DataFrame:
    """
    Makes sure that "timestamp" column has minute resolution with no missing
    values from start to end and replaces all NaNs in "activity" column with
    mean average value.
    
    :param data: DataFrame with "timestamp" and "activity" columns
    :returns: cleaned DataFrame
    """
    df = df.copy()  # create copy to avoid side effects
    
    # resample to the basic frequency, i.e. minute; this will create NaNs for
    # any rows that may be missing
    df = df.resample("min", on="timestamp").mean()
    
    # recreate index and "timestamp" column
    df = df.reset_index()
    
    # fill any NaNs with mean activity value
    df["activity"] = df["activity"].fillna(df["activity"].mean())

    return df


def resample(df: pd.DataFrame, freq: str = "H") -> pd.DataFrame:
    """
    Resamples time series DataFrame with given frequency, aggregating each
    segment with a mean.

    :param df: DataFrame with "timestamp" and "activity" columns
    :param freq: resampling frequency passed to Pandas resample() function
    :returns: DataFrame with "timestamp" and "activity" columns
    """
    df = df.copy()  # create copy to avoid side effects
    
    # group with given frequency
    df = df.resample(freq, on="timestamp").mean()

    # recreate "timestamp" column
    df = df.reset_index()

    return df


def proportion_of_zeros(x: np.ndarray) -> float:
    """
    Calculates proportion of zeros in given array, i.e. number of zeros divided
    by length of array.
    
    :param x: 1D Numpy array
    :returns: proportion of zeros
    """
    # we may be dealing with floating numbers, we can't use direct comparison
    zeros_count = np.sum(np.isclose(x, 0))
    return zeros_count / len(x)


def power_spectral_density(df: pd.DataFrame) -> np.ndarray:
    """
    Calculates power spectral density (PSD) from "activity" column of a
    DataFrame.
    
    :param df: DataFrame with "activity" column
    :returns: 1D Numpy array with power spectral density
    """
    psd = scipy.signal.welch(
        x=df["activity"].values,
        fs=(1/60),
        nperseg=NPERSEG,
        noverlap=NOVERLAP,
        nfft=NFFT,
        window=WINDOW,
        scaling="density"
    )[1]
    return psd


def spectral_flatness(df: pd.DataFrame) -> float:
    """
    Calculates spectral flatness of a signal, i.e. a geometric mean of the
    power spectrum divided by the arithmetic mean of the power spectrum.
    
    If some frequency bins in the power spectrum are close to zero, they are
    removed prior to calculation of spectral flatness to avoid calculation of
    log(0).
    
    :param df: DataFrame with "activity" column
    :returns: spectral flatness value
    """
    power_spectrum = scipy.signal.welch(
        df["activity"].values,
        fs=(1/60),
        nperseg=NPERSEG,
        noverlap=NOVERLAP,
        nfft=NFFT,
        window=WINDOW,
        scaling="spectrum"
    )[1]
    
    non_zeros_mask = ~np.isclose(power_spectrum, 0)
    power_spectrum = power_spectrum[non_zeros_mask]
    
    return scipy.stats.gmean(power_spectrum) / power_spectrum.mean()

## Feature extraction

In [ ]:
def extract_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in time domain.
    
    :param df_resampled: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = df["activity"].values
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X, ddof=1),  # apply Bessel's correction
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "proportion_of_zeros": proportion_of_zeros(X)
    }
    
    return pd.DataFrame([features])

In [ ]:
def extract_frequency_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in frequency domain, i.e. calculated
    from its Power Spectral Density (PSD).
    
    :param df: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = power_spectral_density(df)
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X),
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "spectral_flatness": spectral_flatness(df)
    }
    
    return pd.DataFrame([features])

In [ ]:
def extract_features_for_dataframes(dfs: List[pd.DataFrame], freq: str = "H") \
        -> Dict[str, pd.DataFrame]:
    """
    Calculates time and frequency features for given DataFrames. Uses given
    frequency for resampling.
    
    Calculates features separately for:
    - full 24hs
    - days: [8:00, 21:00)
    - nights: [21:00, 8:00)
    
    :param dfs: list of DataFrames to extract features from; each one has to
    have "timestamp" and "activity" columns
    :param freq: resampling frequency
    :returns: dictionary with keys "full_24h", "day" and "night", corresponding
    to features from given parts of day
    """
    full_dfs = basic_data_cleaning(dfs)
    full_dfs = [fill_missing_activity(df) for df in full_dfs]
    full_dfs = [resample(df, freq=freq) for df in full_dfs]
    
    night_dfs = [get_day_part(df, part="night") for df in full_dfs]
    day_dfs = [get_day_part(df, part="day") for df in full_dfs]

    datasets = {}
    
    for part, list_of_dfs in [("full_24h", full_dfs), ("night", night_dfs),
                              ("day", day_dfs)]:
        features = []
        for df in list_of_dfs:
            time_features = extract_time_features(df)
            freq_features = extract_frequency_features(df)

            merged_features = pd.merge(
                time_features,
                freq_features,
                left_index=True,
                right_index=True,
                suffixes=["_time", "_freq"]
            )
            features.append(merged_features)

        datasets[part] = pd.concat(features)
        datasets[part].reset_index(drop=True, inplace=True)
    
    return datasets

## Depresjon

In [ ]:
dataset = Dataset(dirpath=os.path.join("data", "depresjon"))
condition = dataset.condition
control = dataset.control

In [ ]:
condition[0]

In [ ]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["full_24h", "night", "day"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    
    entire_df = condition_df.append(control_df, ignore_index=True)
    datasets[part] = entire_df

In [ ]:
for part, df in datasets.items():
    filename = f"{DEPRESJON_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [ ]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"depresjon_y.csv")
y.to_csv(filepath, header=False, index=False)

In [ ]:
datasets["full_24h"]

## Psykose

In [ ]:
dataset = Dataset(dirpath=os.path.join("data", "psykose"))
condition = dataset.condition
control = dataset.control

In [ ]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["full_24h", "night", "day"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    
    entire_df = condition_df.append(control_df, ignore_index=True)
    datasets[part] = entire_df

In [ ]:
for part, df in datasets.items():
    filename = f"{PSYKOSE_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [ ]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"psykose_y.csv")
y.to_csv(filepath, header=False, index=False)

# Classification

## Classifiers, parameters, constants

In [ ]:
classifiers = {
    "LR": LogisticRegression(
        penalty="elasticnet",
        random_state=0,
        solver="saga",
        max_iter=5000
    ),
    "SVM": SVC(
        kernel="rbf",
        cache_size=512
    ),
    "RF": RandomForestClassifier(
        n_estimators=500,
        criterion="entropy"
    )
}


param_grids = {
    "LR": {
        "C": [0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10, 25, 50, 100, 500, 1000],
        "class_weight": [None, "balanced"],
        "l1_ratio": [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5,
                     0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
    },
    "SVM": {
        "C": np.logspace(10e-3, 10e3, num=50),
        "gamma": np.logspace(10e-3, 10e3, num=50),
        "class_weight": [None, "balanced"]
    },
    "RF": {
        "class_weight": [None, "balanced", "balanced_subsample"]
    }
}

## Classification

In [ ]:
dataset = PSYKOSE_PREFIX  # DEPRESJON_PREFIX or PSYKOSE_PREFIX
y_filename = "depresjon_y.csv" if dataset == DEPRESJON_PREFIX else "psykose_y.csv"

In [ ]:
datasets = {}

for part in ["full_24h", "night", "day"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()

In [ ]:
for part in ["full_24h", "night", "day"]:
    print(f"PART: {part}")
    
    X = datasets[part]

    for clf_type in ["LR", "SVM", "RF"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="accuracy",
                n_jobs=-1,
                refit=True,
                cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)
            
            clf = grid_search.best_estimator_
            
            metrics = calculate_metrics(clf, X_test, y_test)
            print(metrics)
            test_scores.append(metrics)
        
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()